In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
import time

In [ ]:
def find_sector_ids(url):
    """Function to scrap sector IDs in elements with onclick property from Banxico.

    Parameters
    ----------
    url : str
        banxico url containig the series id information

    Returns
    -------
    sector_ids : list
        list containing sector ids as strings
    """
    sector_ids = []
    try:
        driver = webdriver.Chrome()
        driver.get(url)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(5)
        
        #driver.find_element_by_xpath(".//input[contains(@onclick, '1 Bedroom Deluxe')]")
        elements = driver.find_elements(By.XPATH, '//*[@onclick]')

        if len(elements) < 3: 
            driver.quit()
            time.sleep(10)
            find_sector_ids(url)


        for element_id in elements:
            onclick_txt = element_id.get_attribute('onclick')
            if "cargaDirectorioSector" in onclick_txt:
                sector_ids.append((onclick_txt.split(",")[0].split("(")[-1]))
        return sector_ids

    finally:
        driver.quit()


banxico_series_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries" 

sector_ids = find_sector_ids(banxico_series_url)
sector_ids


['4',
 '23',
 '6',
 '18',
 '22',
 '7',
 '3',
 '19',
 '11',
 '9',
 '8',
 '20',
 '2',
 '10',
 '1',
 '5',
 '21',
 '24',
 '25']